<a href="https://colab.research.google.com/github/yule0707/Lajittelija/blob/main/Oliomuunnos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ### | **HUOM** | Vastaavaa, mutta hieman kevyemmässä mittakaavassa saattaa tulla tenttiin


# Tehtävä: Proseduraalisesta koodista oliopohjaiseksi

## Tavoite
Tässä tehtävässä refaktoroit toimivan mutta "sotkuisen" proseduraalisen Python-ohjelman oliopohjaiseksi (OOP)

### Vaatimukset
- Ohjelman tila (state) **ei saa olla globaaleissa muuttujissa** (vakiot ovat ok).
- Muunna funktiot luokan metodeiksi.
- Säilytä tila olion sisällä `self.`-attribuutteina.
- Ohjelman toiminnallisuuden ja tulosteiden tulee säilyä samana.

### Palautus
- Palauta tämä notebook, jossa OOP-ratkaisu ja testit läpäisevät.


## 1) Lähtötilanne: Proseduraalinen koodi

Aja seuraava solu ja varmista, että näet kuitin tulosteen. Tämä on "baseline", jonka toiminta pitää säilyttää refaktoroinnin jälkeen.

🛑 ÄLÄ TÄYTÄ TÄHÄN KOHTAAN MITÄÄN VAAN KOPIOI TOIMINNALLISUUS KOHTAAN 2

In [2]:
# =========================
# Procedural baseline code
# =========================

# Global "database"
orders = []  # list of dicts: {"id": int, "item": str, "qty": int, "price": float}
next_id = 1
tax_rate = 0.24  # 24% VAT style tax (example)

def add_order(item: str, qty: int, price: float) -> int:
    global next_id, orders
    if qty <= 0:
        raise ValueError("qty must be positive")
    if price < 0:
        raise ValueError("price cannot be negative")
    order = {"id": next_id, "item": item, "qty": qty, "price": float(price)}
    orders.append(order)
    next_id += 1
    return order["id"]

def remove_order(order_id: int) -> bool:
    global orders
    for i, o in enumerate(orders):
        if o["id"] == order_id:
            del orders[i]
            return True
    return False

def update_qty(order_id: int, new_qty: int) -> None:
    global orders
    if new_qty <= 0:
        raise ValueError("new_qty must be positive")
    for o in orders:
        if o["id"] == order_id:
            o["qty"] = new_qty
            return
    raise KeyError(f"Order id {order_id} not found")

def subtotal() -> float:
    return sum(o["qty"] * o["price"] for o in orders)

def total_with_tax() -> float:
    return subtotal() * (1 + tax_rate)

def receipt() -> str:
    lines = ["RECEIPT"]
    for o in orders:
        lines.append(f'#{o["id"]}: {o["item"]} x{o["qty"]} @ {o["price"]:.2f}')
    lines.append(f"Subtotal: {subtotal():.2f}")
    lines.append(f"Total (tax {tax_rate*100:.0f}%): {total_with_tax():.2f}")
    return "\n".join(lines)

def reset_store() -> None:
    global orders, next_id
    orders = []
    next_id = 1

# Demo run (should work)
reset_store()
add_order("Coffee", 2, 3.50)
add_order("Bagel", 1, 2.20)
update_qty(1, 3)
print(receipt())


RECEIPT
#1: Coffee x3 @ 3.50
#2: Bagel x1 @ 2.20
Subtotal: 12.70
Total (tax 24%): 15.75


## 2) Refaktorointitehtävä

Luo luokka nimeltä **`OrderStore`**, joka tarjoaa saman toiminnallisuuden kuin proseduraalinen koodi.

### Pakolliset metodit (sama käyttäytyminen)
- `add_order(item, qty, price) -> int`
- `remove_order(order_id) -> bool`
- `update_qty(order_id, new_qty) -> None`
- `subtotal() -> float`
- `total_with_tax() -> float`
- `receipt() -> str`
- `reset_store() -> None`

### Pakollinen tila (state)
- Tilaukset ja seuraava id tulee säilyttää **instanssiattribuutteina** (`self.orders`, `self.next_id`) eikä globaaleissa muuttujissa.
- `tax_rate` annetaan konstruktorissa `__init__(...)` (oletus 0.24) ja tallennetaan `self.tax_rate`.

### Rajoitteet
- Ei perintää.
- Ei globaalia ohjelmatilaa.
- Voit käyttää tilauksia dict-rakenteena (kuten baseline), tai halutessasi tehdä pienen `Order`-luokan (ilman perintää).


### **Aloitusrunko**

Toteuta metodit yksi kerrallaan. Aja testit usein.

✅ TÄYTÄ ALLE JA EDITOI METODIT

❌ ÄLÄ MUUTA METODIEN NIMIÄ. SE SOTKEE TESTIT.


In [1]:
class OrderStore:
    def __init__(self, tax_rate: float = 0.24):
      self.tax_rate = tax_rate
      self.orders = []
      self.next_id = 1

    def add_order(self, item: str, qty: int, price: float) -> int:

      if qty <= 0:
        raise ValueError("qty must be positive")
      if price < 0:
        raise ValueError("price cannot be negative")

      order = {
            "id": self.next_id,
            "item" : item,
            "qty" : qty,
            "price" : float(price)
        }

      self.orders.append(order)
      self.next_id += 1

      return order["id"]

    def remove_order(self, order_id: int) -> bool:
      for i, o in enumerate(self.orders):
        if o["id"] == order_id:
          del self.orders[i]
          return True
      return False

    def update_qty(self, order_id: int, new_qty: int) -> None:
      if new_qty <= 0:
        raise ValueError("new_qty must be positive")
      for o in self.orders:
        if o["id"] == order_id:
            o["qty"] = new_qty
            return

      raise KeyError(f"Order id {order_id} not found")

    def subtotal(self) -> float:
        return sum(o["qty"] * o["price"] for o in self.orders)

    def total_with_tax(self) -> float:
        return self.subtotal() * (1 + self.tax_rate)

    def receipt(self) -> str:
      lines = ["RECEIPT"]
      for o in self.orders:
        lines.append(f"#{o['id']}: {o['item']} x{o['qty']} @ {o['price']:.2f}")
      lines.append(f"Subtotal: {self.subtotal():.2f}")
      lines.append(f"Total (tax {self.tax_rate*100:.0f}%): {self.total_with_tax():.2f}")
      return "\n".join(lines)

    def reset_store(self) -> None:
        orders = []
        next_id = 1


### Vinkit refaktorointiin

Suositeltu eteneminen:
1. Kopioi logiikka proseduraalisista funktioista vastaaviin metodeihin.
2. Korvaa `global`-muuttujat instanssiattribuuteilla: `self.orders`, `self.next_id`, `self.tax_rate`.
3. Varmista, että poikkeukset (ValueError/KeyError) vastaavat alkuperäistä käyttäytymistä.
4. Aja testit jokaisen metodin jälkeen.


## Testit

Kun olet toteuttanut luokan, seuraavien testien tulee mennä läpi.

❌ ÄLÄ MUUTA TESTEJÄ


In [2]:
import unittest

class TestOrderStore(unittest.TestCase):
    def setUp(self):
        self.s = OrderStore(tax_rate=0.24)
        self.s.reset_store()

    def test_add_and_ids(self):
        a = self.s.add_order("Coffee", 2, 3.5)
        b = self.s.add_order("Bagel", 1, 2.2)
        self.assertEqual(a, 1)
        self.assertEqual(b, 2)
        self.assertAlmostEqual(self.s.subtotal(), 2*3.5 + 1*2.2)

    def test_update_qty(self):
        oid = self.s.add_order("Coffee", 2, 3.5)
        self.s.update_qty(oid, 3)
        self.assertAlmostEqual(self.s.subtotal(), 3*3.5)

    def test_remove(self):
        a = self.s.add_order("Coffee", 2, 3.5)
        _b = self.s.add_order("Bagel", 1, 2.2)
        self.assertTrue(self.s.remove_order(a))
        self.assertFalse(self.s.remove_order(999))
        self.assertAlmostEqual(self.s.subtotal(), 1*2.2)

    def test_errors(self):
        with self.assertRaises(ValueError):
            self.s.add_order("X", 0, 1.0)
        with self.assertRaises(ValueError):
            self.s.add_order("X", 1, -1.0)

        oid = self.s.add_order("Coffee", 1, 1.0)
        with self.assertRaises(ValueError):
            self.s.update_qty(oid, 0)
        with self.assertRaises(KeyError):
            self.s.update_qty(999, 1)

    def test_total_with_tax(self):
        self.s.add_order("Coffee", 2, 10.0)  # subtotal 20
        self.assertAlmostEqual(self.s.total_with_tax(), 20 * 1.24)

    def test_receipt_contains(self):
        self.s.add_order("Coffee", 2, 3.5)
        txt = self.s.receipt()
        self.assertIn("RECEIPT", txt)
        self.assertIn("Coffee", txt)
        self.assertIn("Subtotal:", txt)
        self.assertIn("Total (tax", txt)

suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestOrderStore)
unittest.TextTestRunner(verbosity=2).run(suite)


test_add_and_ids (__main__.TestOrderStore.test_add_and_ids) ... ok
test_errors (__main__.TestOrderStore.test_errors) ... ok
test_receipt_contains (__main__.TestOrderStore.test_receipt_contains) ... ok
test_remove (__main__.TestOrderStore.test_remove) ... ok
test_total_with_tax (__main__.TestOrderStore.test_total_with_tax) ... ok
test_update_qty (__main__.TestOrderStore.test_update_qty) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.013s

OK


<unittest.runner.TextTestResult run=6 errors=0 failures=0>

Tuloksen tulisi näyttää tältä, kun kaikki menee hyvin:


```
test_add_and_ids (__main__.TestOrderStore.test_add_and_ids) ... ok
test_errors (__main__.TestOrderStore.test_errors) ... ok
test_receipt_contains (__main__.TestOrderStore.test_receipt_contains) ... ok
test_remove (__main__.TestOrderStore.test_remove) ... ok
test_total_with_tax (__main__.TestOrderStore.test_total_with_tax) ... ok
test_update_qty (__main__.TestOrderStore.test_update_qty) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.045s

OK
<unittest.runner.TextTestResult run=6 errors=0 failures=0>
```

